In [ ]:
# https://howlongtobeat.com/
# based on https://github.com/ScrappyCocco/HowLongToBeat-PythonAPI
# Février 2022
# sortie = Title name | Main Story duration

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import requests
from time import sleep
import re

In [ ]:
user_agent = {'User-agent': 'Mozilla/5.0'}
BASE_URL = 'https://howlongtobeat.com/'
SEARCH_URL = BASE_URL + "search_results.php"

In [ ]:
def recherche_LH2B(name):
    payload = {
        'queryString': name,
        't': 'games',
        'sorthead': 'popular',
        'sortd': 'Normal Order',
        'plat': '',
        'length_type': 'main',
        'length_min': '',
        'length_max': '',
        'detail': '' }
    
    return requests.post(SEARCH_URL, data=payload, headers=user_agent)

In [ ]:
vgchartz = pd.read_csv("vgsales_at.csv", sep=",")
print (vgchartz.shape)
vgchartz.head()

In [ ]:
vgchartz["title_norm"] = vgchartz["Name"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
vgchartz["title_norm"] = vgchartz["title_norm"].str.upper()
vgchartz.head()

In [ ]:
list_jeux_df = vgchartz.Name.unique()
print (len(list_jeux_df))
list_jeux_df[:10]

In [ ]:
games_dict = {}

for name in tqdm(list_jeux_df[:]):

    if name in games_dict.keys() :   # on la dejà récupéré
        continue                     # on zappe
    
    response = recherche_LH2B(name)
    soup = BeautifulSoup(response.text, 'html.parser')

    #find all the games of the request
    games = soup.select(".search_list_details")

    # recherche  de la 'main story' de notre jeux
    for g in range(len(games)):
        # on va recup tous les jeux d'une requette

        game_name = games[g].findAll(name = 'a')[0].text.strip()

        # Pour chaque jeux on cherche la 'Main Story'
        trouve = False
        for detail in games[g].findAll(name = 'div'):
            t = detail.text.strip()
            if trouve:
                games_dict[game_name] = t
                break
            if t == 'Main Story':   # ah ah, on a un chiffre à la ligne suivante !
                trouve = True

    sleep(0.5) # pas trop vite ;)
   

In [ ]:
df = pd.DataFrame.from_dict(games_dict,orient='index',columns=['Main story duration']).reset_index().rename(columns={'index':"Name"})
df.head()

In [ ]:
df.shape

In [ ]:
df['Main story duration'] = df['Main story duration'].str.replace("½",".5")
df[['duration', 'unit']] = df['Main story duration'].str.extract(r'(\d+\.?\d?) +(\w+)')
df['duration'] = df['duration'].astype('float64')
df['Duration hrs'] = df.apply(lambda r: r['duration'] if r['unit']=='Hours' else r['duration']/60, axis=1)
df.drop(columns=['duration','unit'], inplace=True)
df.head(5)

In [ ]:
df.to_csv("hl2b_sl.csv", sep=",", encoding='utf-8', index=False)

In [ ]:
df.tail(5)

In [ ]:
(df['Main story duration']=='--').value_counts()

In [ ]:
df['Duration hrs'].isna().value_counts()